# EDA

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import altair as alt
from IPython.display import display
pd.options.display.max_columns = None # such that jupyter prints out all columns and rows
pd.options.display.max_rows = None

In [11]:
# load data file here
df = pd.read_csv('../data/player_data_ready.csv')

In [12]:
# make primary key column
df['pk'] = df["playDispNm"] + '_' + df["gmDate"]

#### Remove Unnecessary Columns

In [13]:
info_cols = ['pk', 'playDispNm', 'gmDate', 'teamAbbr']
df_info = df[info_cols]

df = df.drop(columns=info_cols)

#### Train/test split

In [14]:
df_train, df_test = train_test_split(df, test_size=0.2)

#### Start basic checks

In [15]:
df_train.describe()

,playMin,teamLoc,playStat,teamLoc_last1_mean,teamLoc_last5_mean,teamLoc_last5_std,teamLoc_last5_median,teamLoc_last10_mean,teamLoc_last10_std,teamLoc_last10_median,teamLoc_last15_mean,teamLoc_last15_std,teamLoc_last15_median,teamLoc_last20_mean,teamLoc_last20_std,teamLoc_last20_median,teamLoc_last25_std,teamLoc_last30_std,teamRslt_last1_mean,teamRslt_last5_mean,teamRslt_last5_std,teamRslt_last5_median,teamRslt_last10_mean,teamRslt_last10_std,teamRslt_last10_median,teamRslt_last15_mean,teamRslt_last15_std,teamRslt_last15_median,teamRslt_last20_mean,teamRslt_last20_std,teamRslt_last20_median,teamRslt_last25_std,teamRslt_last30_std,playMin_last1_mean,playMin_last5_mean,playMin_last5_std,playMin_last5_median,playMin_last10_mean,playMin_last10_std,playMin_last10_median,playMin_last15_mean,playMin_last15_std,playMin_last15_median,playMin_last20_mean,playMin_last20_std,playMin_last20_median,playMin_last25_std,playMin_last30_std,playPTS_last1_mean,playPTS_last5_mean,playPTS_last5_std,playPTS_last5_median,playPTS_last10_mean,playPTS_last10_std,playPTS_last10_median,playPTS_last15_mean,playPTS_last15_std,playPTS_last15_median,playPTS_last20_mean,playPTS_last20_std,playPTS_last20_median,playPTS_last25_std,playPTS_last30_std,playAST_last1_mean,playAST_last5_mean,playAST_last5_std,playAST_last5_median,playAST_last10_mean,playAST_last10_std,playAST_last10_median,playAST_last15_mean,playAST_last15_std,playAST_last15_median,playAST_last20_mean,playAST_last20_std,playAST_last20_median,playAST_last25_std,playAST_last30_std,playTO_last1_mean,playTO_last5_mean,playTO_last5_std,playTO_last5_median,playTO_last10_mean,playTO_last10_std,playTO_last10_median,playTO_last15_mean,playTO_last15_std,playTO_last15_median,playTO_last20_mean,playTO_last20_std,playTO_last20_median,playTO_last25_std,playTO_last30_std,playSTL_last1_mean,playSTL_last5_mean,playSTL_last5_std,playSTL_last5_median,playSTL_last10_mean,playSTL_last10_std,playSTL_last10_median,playSTL_last15_mean,playSTL_last15_std,playSTL_last15_median,playSTL_last20_mean,playSTL_last20_std,playSTL_last20_median,playSTL_last25_std,playSTL_last30_std,playBLK_last1_mean,playBLK_last5_mean,playBLK_last5_std,playBLK_last5_median,playBLK_last10_mean,playBLK_last10_std,playBLK_last10_median,playBLK_last15_mean,playBLK_last15_std,playBLK_last15_median,playBLK_last20_mean,playBLK_last20_std,playBLK_last20_median,playBLK_last25_std,playBLK_last30_std,playTRB_last1_mean,playTRB_last5_mean,playTRB_last5_std,playTRB_last5_median,playTRB_last10_mean,playTRB_last10_std,playTRB_last10_median,playTRB_last15_mean,playTRB_last15_std,playTRB_last15_median,playTRB_last20_mean,playTRB_last20_std,playTRB_last20_median,playTRB_last25_std,playTRB_last30_std,teamLoc_ewm_01,teamLoc_ewm_02,teamLoc_ewm_025,teamLoc_ewm_03,teamLoc_ewm_04,teamLoc_ewm_05,teamLoc_ewm_06,teamLoc_ewm_07,teamRslt_ewm_01,teamRslt_ewm_02,teamRslt_ewm_025,teamRslt_ewm_03,teamRslt_ewm_04,teamRslt_ewm_05,teamRslt_ewm_06,teamRslt_ewm_07,playMin_ewm_01,playMin_ewm_02,playMin_ewm_025,playMin_ewm_03,playMin_ewm_04,playMin_ewm_05,playMin_ewm_06,playMin_ewm_07,playPTS_ewm_01,playPTS_ewm_02,playPTS_ewm_025,playPTS_ewm_03,playPTS_ewm_04,playPTS_ewm_05,playPTS_ewm_06,playPTS_ewm_07,playAST_ewm_01,playAST_ewm_02,playAST_ewm_025,playAST_ewm_03,playAST_ewm_04,playAST_ewm_05,playAST_ewm_06,playAST_ewm_07,playTO_ewm_01,playTO_ewm_02,playTO_ewm_025,playTO_ewm_03,playTO_ewm_04,playTO_ewm_05,playTO_ewm_06,playTO_ewm_07,playSTL_ewm_01,playSTL_ewm_02,playSTL_ewm_025,playSTL_ewm_03,playSTL_ewm_04,playSTL_ewm_05,playSTL_ewm_06,playSTL_ewm_07,playBLK_ewm_01,playBLK_ewm_02,playBLK_ewm_025,playBLK_ewm_03,playBLK_ewm_04,playBLK_ewm_05,playBLK_ewm_06,playBLK_ewm_07,playTRB_ewm_01,playTRB_ewm_02,playTRB_ewm_025,playTRB_ewm_03,playTRB_ewm_04,playTRB_ewm_05,playTRB_ewm_06,playTRB_ewm_07
count,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,1

> * The target feature `playMin` is showing a reasonable statistical summary.  
> * Intuitively important explanatory features like `playMin_last"n"`, `playStat`, `playPTS_last"n"` are showing reasonable statistical summaries as well.

In [16]:
# check missing values
assert df_train.isna().sum().sum() == 0, "There shouldn't be any missing values!"

> There's no missing value in the data (because I've dropped them in the wrangling step).

#### Get the Correlation between all explanatory variables and target

In [17]:
correlations = {}
for col in df:
    if col == 'playMin':
        continue
    try:
        correlations[col] = round(np.corrcoef(df[col], df['playMin'])[0][1], 3)
    except:
        continue
        
correl_df = pd.DataFrame.from_dict(correlations, orient='index')
correl_df.columns = ['corr w/ target']
correl_df = correl_df.sort_values('corr w/ target', ascending=False)
print("""Top pos/neg correlated features against the target: \n{}\n\n{}\n""".format(correl_df.head(20), correl_df.tail(8)))

Top pos/neg correlated features against the target: 
                       corr w/ target
playMin_ewm_03                  0.782
playMin_ewm_025                 0.782
playMin_ewm_02                  0.781
playMin_ewm_04                  0.779
playMin_ewm_05                  0.773
playMin_ewm_01                  0.769
playMin_last5_mean              0.769
playMin_ewm_06                  0.765
playMin_last10_mean             0.762
playMin_last5_median            0.756
playMin_ewm_07                  0.755
playMin_last15_mean             0.754
playMin_last10_median           0.753
playMin_last20_mean             0.746
playMin_last15_median           0.744
playMin_last20_median           0.737
playMin_last1_mean              0.713
playPTS_ewm_02                  0.674
playPTS_ewm_01                  0.671
playPTS_ewm_025                 0.671

                    corr w/ target
teamRslt_ewm_07             -0.005
teamRslt_ewm_06             -0.005
playMin_last5_std           -0.085
playMin_

In [18]:
# visualize the correlations
sort = list(correl_df.reset_index()['index'])
alt.Chart(correl_df.reset_index()).mark_line().encode(
    x=alt.X('index', sort=sort),
    y='corr w/ target:Q'
).properties(
    width=1200,
    height=200
)

alt.Chart(...)

![plot](../img/01-EAD_visualization.png)

> * The plot above doesn't render properly on GitHub. So I added it as a picture again.   
> * As expected, features like `playMin_last"n"` have very high correlation with the target. `playPTS` features also have high correlations, which is a bit surprising.  
> * We might consider enlarge "n" for `playMin_last"n"_std` features.  
> * Our base model to beat might be the `playMin_last_5_mean` feature.